In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Загрузка данных
data = pd.read_excel('/content/turleri_onirler_men_zhyldar_b7-v12.xlsx')

# Очистка данных: удаление пробелов и преобразование в числовой формат
resp_2022 = pd.to_numeric(data['Болезни органов дыхания 2022 год'].str.replace(' ', ''), errors='coerce').dropna()
resp_2023 = pd.to_numeric(data['Болезни органов дыхания 2023 год'].str.replace(' ', ''), errors='coerce').dropna()

# Объединение данных в один DataFrame для построения boxplot
combined_data = pd.DataFrame({
    'Заболеваемость': pd.concat([resp_2022, resp_2023], ignore_index=True),
    'Год': ['2022'] * len(resp_2022) + ['2023'] * len(resp_2023)
})

# Построение ящиков с усами
plt.figure(figsize=(10, 6))
sns.boxplot(x='Год', y='Заболеваемость', data=combined_data)
plt.title('Сравнение заболеваемости органов дыхания (2022 vs 2023)')
plt.xlabel('Год')
plt.ylabel('Заболеваемость (на 100 тыс.)')

# Отображение графика
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

# Загрузка данных и очистка
data = pd.read_excel('/turleri_onirler_men_zhyldar_b7-v12.xlsx')
resp_2022 = pd.to_numeric(data['Болезни органов дыхания 2022 год'].str.replace(' ', ''), errors='coerce').dropna()

# 1. Случайная выборка (размер 10)
sample1 = resp_2022.sample(10, random_state=42)
print("Случайная выборка 1:")
print(sample1)

# Характеристики выборки 1
mean1 = sample1.mean()
median1 = sample1.median()
var1 = sample1.var()
std1 = sample1.std()

print(f"\nВыборка 1 - Среднее: {mean1}, Медиана: {median1}, Дисперсия: {var1}, Стандартное отклонение: {std1}")

# 2. Бутстрап-метод для построения выборочного распределения средних
n_bootstraps = 1000
bootstrap_means = [resp_2022.sample(10, replace=True).mean() for _ in range(n_bootstraps)]

# Построение гистограммы бутстрап-распределения
plt.figure(figsize=(10, 6))
sns.histplot(bootstrap_means, kde=True, bins=30)
plt.title('Бутстрап-распределение средних значений (выборка 2022)')
plt.xlabel('Среднее значение')
plt.ylabel('Частота')

# Доверительный интервал (95%)
ci_lower = np.percentile(bootstrap_means, 2.5)
ci_upper = np.percentile(bootstrap_means, 97.5)
print(f"\nДоверительный интервал для среднего (95%): [{ci_lower}, {ci_upper}]")

# 3. Вторая случайная выборка для сравнения (размер 10)
sample2 = resp_2022.sample(10, random_state=24)
print("\nСлучайная выборка 2:")
print(sample2)

# Характеристики выборки 2
mean2 = sample2.mean()
var2 = sample2.var()

print(f"\nВыборка 2 - Среднее: {mean2}, Дисперсия: {var2}")

# Сравнение выборок по средним и дисперсиям
print(f"\nРазница в средних: {abs(mean1 - mean2)}")
print(f"Разница в дисперсиях: {abs(var1 - var2)}")

# Отображение графика
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import kendalltau, spearmanr
import warnings

# Подавление предупреждений openpyxl
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

# Загрузка данных
data = pd.read_excel('/turleri_onirler_men_zhyldar_b7-v12.xlsx')

# Функция для очистки столбцов: удаление пробелов и преобразование в числа
def clean_column(column):
    if column.dtype == 'object':
        return pd.to_numeric(column.str.replace(' ', ''), errors='coerce')
    return column

# Применяем очистку к числовым признакам
data = data.apply(clean_column)

# 1. Коэффициенты корреляции Кендалла и Спирмена
feature1 = data['Болезни органов дыхания 2022 год'].dropna()
feature2 = data['Болезни органов дыхания 2023 год'].dropna()

# Вычисление корреляций
kendall_corr, _ = kendalltau(feature1, feature2)
spearman_corr, _ = spearmanr(feature1, feature2)

print(f"Коэффициент корреляции Кендалла: {kendall_corr}")
print(f"Коэффициент корреляции Спирмена: {spearman_corr}")

# 2. Логарифмирование и пересчёт корреляций
log_feature1 = np.log1p(feature1)
log_feature2 = np.log1p(feature2)

kendall_corr_log, _ = kendalltau(log_feature1, log_feature2)
spearman_corr_log, _ = spearmanr(log_feature1, log_feature2)

print(f"\nПосле логарифмирования:")
print(f"Коэффициент Кендалла: {kendall_corr_log}")
print(f"Коэффициент Спирмена: {spearman_corr_log}")

# 3. Корреляционная матрица и тепловая карта
corr_matrix = data.corr(method='spearman')

plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Тепловая карта корреляций (Спирмен)')
plt.show()


In [ ]:
# Загрузка и очистка данных
data = pd.read_excel('/turleri_onirler_men_zhyldar_b7-v12.xlsx')

# Очистка столбцов с числовыми значениями
def clean_column(column):
    if column.dtype == 'object':
        return pd.to_numeric(column.str.replace(' ', ''), errors='coerce')
    return column

data = data.apply(clean_column)

# Очистка и проверка столбца "Регион"
if 'Регион' in data.columns:
    data['Регион'] = data['Регион'].astype(str).str.strip()  # Преобразуем в строку и очищаем пробелы

# Проверка уникальных значений в столбце "Регион"
print("Уникальные регионы:")
print(data['Регион'].unique())

# 1. Проверка гипотезы о нормальности распределения (Андерсон-Дарлинг и Колмогоров-Смирнов)
feature1 = data['Болезни органов дыхания 2022 год'].dropna()

# Тест Андерсона-Дарлинга
result_ad = anderson(feature1)
print("Тест Андерсона-Дарлинга:")
print(f"Статистика: {result_ad.statistic}, Критические значения: {result_ad.critical_values}")

# Критерий Колмогорова-Смирнова
stat_ks, p_ks = kstest(feature1, 'norm', args=(feature1.mean(), feature1.std()))
print(f"\nКритерий Колмогорова-Смирнова: Статистика = {stat_ks}, p-значение = {p_ks}")

# 2. Проверка равенства средних с помощью U-критерия Манна-Уитни
feature2 = data['Болезни органов дыхания 2023 год'].dropna()

stat_mw, p_mw = mannwhitneyu(feature1, feature2, alternative='two-sided')
print(f"\nU-критерий Манна-Уитни: Статистика = {stat_mw}, p-значение = {p_mw}")

# 3. Анализ мощности теста
effect_size = (feature1.mean() - feature2.mean()) / np.sqrt((feature1.var() + feature2.var()) / 2)
power_analysis = TTestIndPower()
power = power_analysis.solve_power(effect_size, nobs1=len(feature1), alpha=0.05)
print(f"\nМощность теста: {power:.3f}")



In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.decomposition import FactorAnalysis
import matplotlib.pyplot as plt
import seaborn as sns

# Загрузка данных из JSON файла
df = pd.read_json('/turleri_onirler_men_zhyldar_b7-v12.json')

# Выбираем числовые колонки для анализа
numeric_columns = ['tub', 'obshzab', 'travmi', 'travm', 'bolsistkrov',
                  'zlokazh', 'obshsabl', 'bod', 'bsk', 'zlo', 'bodix']
df_numeric = df[numeric_columns].copy()

# Функция для преобразования строк в числа
def convert_to_float(x):
    if isinstance(x, str):
        return float(x.replace(' ', ''))
    return float(x)

# Применяем преобразование к каждому числовому столбцу
for column in numeric_columns:
    df_numeric[column] = df_numeric[column].apply(convert_to_float)

# Очистка данных от выбросов и некорректных значений
df_numeric = df_numeric.replace([45377, 45406, 45439, 45465, 45498, 45535], np.nan)
df_numeric = df_numeric.dropna()

# Масштабирование данных
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_numeric), columns=df_numeric.columns)

# 1. DBSCAN кластеризация
dbscan = DBSCAN(eps=0.5, min_samples=2)
clusters = dbscan.fit_predict(df_scaled)

# Визуализация результатов DBSCAN
plt.figure(figsize=(10, 6))
plt.scatter(df_scaled['obshzab'], df_scaled['bod'], c=clusters, cmap='viridis')
plt.title('DBSCAN кластеризация')
plt.xlabel('Общая заболеваемость (масштабированная)')
plt.ylabel('Болезни органов дыхания (масштабированная)')
plt.colorbar(label='Кластер')
plt.savefig('dbscan_clusters.png')
plt.close()

# 2. Регрессионное дерево
X = df_numeric.drop('bod', axis=1)  # Предикторы
y = df_numeric['bod']  # Целевая переменная

tree_model = DecisionTreeRegressor(max_depth=3, random_state=42)
tree_model.fit(X, y)

# Визуализация дерева решений
plt.figure(figsize=(15, 10))
plot_tree(tree_model, feature_names=X.columns, filled=True, rounded=True)
plt.savefig('decision_tree.png')
plt.close()

# Важность признаков
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': tree_model.feature_importances_
}).sort_values('importance', ascending=False)

# 3. Факторный анализ
fa = FactorAnalysis(n_components=3, random_state=42)
fa_transformed = fa.fit_transform(df_scaled)
components = fa.components_

# Создание DataFrame для компонент
components_df = pd.DataFrame(
    components,
    columns=df_scaled.columns,
    index=['Фактор 1', 'Фактор 2', 'Фактор 3']
)

# Тепловая карта компонент
plt.figure(figsize=(12, 8))
sns.heatmap(components_df, annot=True, cmap='coolwarm', center=0)
plt.title('Факторные нагрузки')
plt.tight_layout()
plt.savefig('factor_analysis.png')
plt.close()

# Вывод результатов анализа
results = {
    'DBSCAN': {
        'n_clusters': len(set(clusters)) - (1 if -1 in clusters else 0),
        'n_noise': list(clusters).count(-1)
    },
    'Feature Importance': feature_importance.to_dict(),
    'Factor Analysis': {
        'explained_variance': (np.var(fa_transformed, axis=0) / np.var(fa_transformed, axis=0).sum() * 100).tolist()
    }
}

print("\nРезультаты DBSCAN кластеризации:")
print(f"Количество кластеров: {results['DBSCAN']['n_clusters']}")
print(f"Количество шумовых точек: {results['DBSCAN']['n_noise']}")

print("\nВажность признаков в регрессионном дереве:")
print(feature_importance)

print("\nОбъясненная дисперсия факторным анализом (%):")
for i, var in enumerate(results['Factor Analysis']['explained_variance'], 1):
    print(f"Фактор {i}: {var:.2f}%")

# Сохранение результатов в файл
with open('analysis_results.txt', 'w', encoding='utf-8') as f:
    f.write("Результаты статистического анализа\n")
    f.write("==================================\n\n")

    f.write("1. DBSCAN кластеризация\n")
    f.write(f"Количество кластеров: {results['DBSCAN']['n_clusters']}\n")
    f.write(f"Количество шумовых точек: {results['DBSCAN']['n_noise']}\n\n")

    f.write("2. Важность признаков в регрессионном дереве\n")
    f.write(feature_importance.to_string())
    f.write("\n\n")

    f.write("3. Объясненная дисперсия факторным анализом (%)\n")
    for i, var in enumerate(results['Factor Analysis']['explained_variance'], 1):
        f.write(f"Фактор {i}: {var:.2f}%\n")


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import plotly.express as px

# Загрузка и очистка данных
data = pd.read_excel('/content/turleri_onirler_men_zhyldar_b7-v12.xlsx')

# Очистка числовых столбцов
def clean_column(column):
    if column.dtype == 'object':
        return pd.to_numeric(column.str.replace(' ', ''), errors='coerce')
    return column

data = data.apply(clean_column)

# Список количественных переменных для анализа
quantitative_columns = [
    'Болезни органов дыхания 2022 год',
    'Болезни органов дыхания 2023 год',
    'Общая заболеваемость 2022 год',
    'Общая заболеваемость 2023 год'
]

# 1. Плотностные графики (density plot) для нескольких количественных переменных
plt.figure(figsize=(12, 8))
for col in quantitative_columns:
    sns.kdeplot(data[col].dropna(), label=col)
plt.title('Плотностные графики для количественных переменных')
plt.xlabel('Значение')
plt.ylabel('Плотность')
plt.legend()
plt.show()

# 2. Матрица корреляции с тепловой картой (heatmap)
correlation_matrix = data[quantitative_columns].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Корреляционная матрица количественных признаков')
plt.show()

# 3. Трёхмерный scatter plot для визуализации кластеризации
# Выполним кластеризацию с помощью KMeans для наглядности
kmeans = KMeans(n_clusters=3, random_state=42)
data['Cluster'] = kmeans.fit_predict(data[quantitative_columns].fillna(0))  # Заполнение NaN нулями для кластеризации

fig = px.scatter_3d(data,
                    x='Болезни органов дыхания 2022 год',
                    y='Болезни органов дыхания 2023 год',
                    z='Общая заболеваемость 2022 год',
                    color='Cluster',
                    title='3D Scatter Plot для кластеризации')
fig.show()

# 4. Парные диаграммы рассеяния (pairplot) для визуализации взаимосвязей
sns.pairplot(data[quantitative_columns].dropna())
plt.suptitle('Парные диаграммы рассеяния для количественных переменных', y=1.02)
plt.show()